# Gathering Data and Generating Images for Classification

## Gather Cryptocurrency Data from Exchange APIs

In [1]:
# TODO?

## Generate Images from Data

In [2]:
from IPython.display import FileLink, FileLinks


In [10]:
import os
import shutil
import sys
from time import time
from uuid import uuid4

import numpy as np
import pandas as pd

from data_manager import file_processor
# from returns_quantization import add_returns_in_place
# from utils import *
import datetime
import matplotlib


In [11]:
# np.set_printoptions(threshold=np.nan)
# pd.set_option('display.height', 1000)
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

matplotlib.use('Agg')

Lets break this down...

# Functions

In [12]:
def compute_returns(p):
    close_prices = p['price_close']
    close_prices_returns = 100 * ((close_prices.shift(-1) - close_prices) / close_prices).fillna(0.0)
    return close_prices_returns.shift(1).fillna(0)

In [13]:
def plot_p(df):
    import matplotlib.pyplot as plt
    from matplotlib.finance import candlestick2_ohlc
    fig, ax = plt.subplots()
    candlestick2_ohlc(ax,
                      df['price_open'].values,
                      df['price_high'].values,
                      df['price_low'].values,
                      df['price_close'].values,
                      width=0.6,
                      colorup='g',
                      colordown='r',
                      alpha=1)
    plt.show()
    print('Done.')

In [14]:
def save_to_file(df, filename):
    import matplotlib.pyplot as plt
    from matplotlib.finance import candlestick2_ohlc
    fig, ax = plt.subplots()
    candlestick2_ohlc(ax,
                      df['price_open'].values,
                      df['price_high'].values,
                      df['price_low'].values,
                      df['price_close'].values,
                      width=0.6,
                      colorup='g',
                      colordown='r',
                      alpha=1)
    plt.savefig(filename)
    plt.close(fig)

In [16]:
def mkdir_p(path):
    import os
    import errno
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

# Process Tick Data

In [15]:
data_file = './data/coinbaseUSD.csv'
data_output_folder = './data/btcgraphs/'

### This is def file_processor Fucntion
def file_processor(data_file):

In [6]:
print('Reading bitcoin market data file here: {}.'.format(data_file))

# create df from tick data
# [unix timestamp, price, volume]
# use the timestamp as the index
d = pd.read_table(data_file, sep=',', header=None, index_col=0, names=['price', 'volume'])

# map the index to datetime
d.index = d.index.map(lambda ts: datetime.datetime.fromtimestamp(int(ts)))
d.index.names = ['DateTime_UTC']

# split the prices into 5 minute groups 
p = pd.DataFrame(d['price'].resample('5Min').ohlc())
p.columns = ['price_open', 'price_high', 'price_low', 'price_close']

# sum volume by 5 minute chunks
v = pd.DataFrame(d['volume'].resample('5Min').sum())
v.columns = ['volume']
p['volume'] = v['volume']

# drop NaN values.
p = p.dropna()
print('Done')

Reading bitcoin market data file here: ./data/coinbaseUSD.csv.
Done


In [7]:
# choosing everything starting after 2015.... no data for first 6 days unfortunately... might need to find new data source
p = p.loc[p.index >= datetime.datetime(2015,1,1,0,0,0)]
p.head(n=5)

,price_open,price_high,price_low,price_close,volume
DateTime_UTC,,,,,
2015-01-07 20:20:00,360.00,360.00,360.00,360.00,0.01
2015-01-07 20:25:00,271.84,344.67,271.60,344.67,0.06
2015-01-07 20:30:00,346.41,346.41,270.35,321.98,0.13
2015-01-07 20:35:00,302.25,349.67,281.36,341.35,0.11
2015-01-07 20:40:00,348.77,351.38,273.20,351.38,0.11


# Generate the Data

### This is the generate_cnn_dataset function
def generate_cnn_dataset(data_folder, bitcoin_file, get_class_name):

In [17]:
data_folder = data_output_folder

In [19]:
# compute_returns(p)
close_prices = p['price_close']
close_prices_returns = 100 * ((close_prices.shift(-1) - close_prices) / close_prices).fillna(0.0)
close_prices_returns = close_prices_returns.shift(1).fillna(0)
close_prices_returns.head(n=5)

DateTime_UTC
2015-01-07 20:20:00    0.000000
2015-01-07 20:25:00   -4.258333
2015-01-07 20:30:00   -6.583108
2015-01-07 20:35:00    6.015902
2015-01-07 20:40:00    2.938333
Name: price_close, dtype: float64

In [ ]:
# def add_returns_in_place(p):  
# close_prices_returns = compute_returns(p)
num_bins = 10
returns_bins = pd.qcut(close_prices_returns, num_bins)
bins_categories = returns_bins.values.categories
returns_labels = pd.qcut(close_prices_returns, num_bins, labels=False)

p['close_price_returns'] = close_prices_returns
p['close_price_returns_bins'] = returns_bins
p['close_price_returns_labels'] = returns_labels

In [25]:
bins_categories

IntervalIndex([(-42.016, -0.185], (-0.185, -0.0749], (-0.0749, -0.0298], (-0.0298, -0.00504], (-0.00504, 0.0], (0.0, 0.0131], (0.0131, 0.0394], (0.0394, 0.085], (0.085, 0.19], (0.19, 67.889]]
              closed='right',
              dtype='interval[float64]')

In [21]:
p.head(n=20)
# return df, bins_categories

,price_open,price_high,price_low,price_close,volume,close_price_returns,close_price_returns_bins,close_price_returns_labels
DateTime_UTC,,,,,,,,
2015-01-07 20:20:00,360.00,360.00,360.00,360.00,0.01,0.000000,"(-0.00504, 0.0]",4
2015-01-07 20:25:00,271.84,344.67,271.60,344.67,0.06,-4.258333,"(-42.016, -0.185]",0
2015-01-07 20:30:00,346.41,346.41,270.35,321.98,0.13,-6.583108,"(-42.016, -0.185]",0
2015-01-07 20:35:00,302.25,349.67,281.36,341.35,0.11,6.015902,"(0.19, 67.889]",9
2015-01-07 20:40:00,348.77,351.38,273.20,351.38,0.11,2.938333,"(0.19, 67.889]",9
2015-01-07 20:45:00,307.38,359.39,279.37,301.99,0.14,-14.056008,"(-42.016, -0.185]",0
2015-01-07 20:50:00,303.53,333.45,272.63,331.94,0.11,9.917547,"(0.19, 67.889]",9
2015-01-07 20:55:00,297.55,343.03,283.25,283.25,0.12,-14.668314,"(-42.016, -0.185]",0
2015-01-07 21:00:00,334.27,344.94,279.55,333.68,0.11,17.804060,"(0.19, 67.889]",9


In [26]:
# btc_df, levels = add_returns_in_place(btc_df)
levels = bins_categories

print('-' * 80)
print('Those values should be roughly equal to 1/len(levels):')
for ii in range(len(levels)):
    print(ii, np.mean((btc_df['close_price_returns_labels'] == ii).values))
print(levels)
print('-' * 80)

--------------------------------------------------------------------------------
Those values should be roughly equal to 1/len(levels):
0 0.10000030679646202
1 0.10000030679646202
2 0.10000030679646202
3 0.10000337476108226
4 0.10731740241571534
5 0.09268014321258847
6 0.10000030679646202
7 0.09999723883184179
8 0.10000030679646202
9 0.10000030679646202
IntervalIndex([(-42.016, -0.185], (-0.185, -0.0749], (-0.0749, -0.0298], (-0.0298, -0.00504], (-0.00504, 0.0], (0.0, 0.0131], (0.0131, 0.0394], (0.0394, 0.085], (0.085, 0.19], (0.19, 67.889]]
              closed='right',
              dtype='interval[float64]')
--------------------------------------------------------------------------------


In [28]:
def get_price_direction(btc_df, btc_slice, i, slice_size):
    last_price = btc_slice[-2:-1]['price_close'].values[0]
    next_price = btc_df[i + slice_size:i + slice_size + 1]['price_close'].values[0]
    if last_price < next_price:
        class_name = 'UP'
    else:
        class_name = 'DOWN'
    return class_name

In [29]:
# number of periods in our input samples
slice_size = 40
# 1/10 data "chuncks" will be for testing
test_every_steps = 10
# number of 5-minute periods we are creating chunks from, 
# need to not start chunk within last 40 or will run out of space
n = len(btc_df) - slice_size

shutil.rmtree(data_folder, ignore_errors=True)

# this is the number of samples we are going to make from the data
cycles = 1e6 

In [ ]:
for epoch in range(int(cycles)):
    st = time()
    
    # choose a random starting point
    i = np.random.choice(n)
    # take following 40 time periods (total 41)
    btc_slice = btc_df[i:i + slice_size]

    if btc_slice.isnull().values.any():
        # sometimes prices are discontinuous and nothing happened in one 5min bucket.
        # in that case, we consider this slice as wrong and we raise an exception.
        # it's likely to happen at the beginning of the data set where the volumes are low.
        raise Exception('NaN values detected. Please remove them.')

    class_name = get_price_direction(btc_df, btc_slice, i, slice_size)
    save_dir = os.path.join(data_folder, 'train', class_name)
    if epoch % test_every_steps == 0:
        save_dir = os.path.join(data_folder, 'test', class_name)
    mkdir_p(save_dir)
    filename = save_dir + '/' + str(uuid4()) + '.png'
    save_to_file(btc_slice, filename=filename)
    print('epoch = {0}, time = {1:.3f}, filename = {2}'.format(str(epoch).zfill(8), time() - st, filename))